In [17]:
import sys
sys.path.append(r"D:\Recyclogy\Own models\tf-TabNet")

from tabnet.tabnet import TabNetClassifier

print("TensorFlow TabNet is ready!")


TensorFlow TabNet is ready!


In [ ]:
# ✅ Imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf
import sys

# Add tf-TabNet path
sys.path.append(r"D:\Recyclogy\Own models\tf-TabNet")
from tabnet.tabnet import TabNetClassifier

# ⚡ Check GPU
gpus = tf.config.list_physical_devices('GPU')
print("Num GPUs Available:", len(gpus))
print("GPUs:", gpus)

# 📊 Load Dataset
df = pd.read_csv('waste_product_v1.csv')

# 🧱 Preprocessing
target_encoder = LabelEncoder()
df['Project_Label'] = target_encoder.fit_transform(df['Project'])

df['Waste_Items_List'] = df['Waste Items'].apply(lambda x: [item.strip() for item in x.split(',')])
df['Tools_List'] = df['Tools'].apply(lambda x: [item.strip() for item in x.split(',')])

mlb_waste = MultiLabelBinarizer()
mlb_tools = MultiLabelBinarizer()

waste_features = mlb_waste.fit_transform(df['Waste_Items_List'])
tools_features = mlb_tools.fit_transform(df['Tools_List'])

X = np.hstack([waste_features, tools_features])
y = df['Project_Label'].values

# Split dataset
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Convert features to dict format
def array_to_dict(X_array):
    return {str(i): X_array[:, i] for i in range(X_array.shape[1])}

X_train_dict = array_to_dict(X_train)
X_valid_dict = array_to_dict(X_valid)

# TabNet feature columns
input_dim = X_train.shape[1]
feature_columns = [tf.feature_column.numeric_column(str(i)) for i in range(input_dim)]

# Initialize TabNet
num_classes = len(np.unique(y_train))
tabnet_model = TabNetClassifier(
    feature_columns=feature_columns,
    num_classes=num_classes,
    feature_dim=64,
    output_dim=32,
    num_decision_steps=5,
    relaxation_factor=1.5
)

# ✅ Compile the model (required!)
tabnet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-2),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# ⏱ Train
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

tabnet_model.fit(
    X_train_dict, y_train,
    validation_data=(X_valid_dict, y_valid),
    batch_size=1024,
    epochs=100,
    callbacks=[early_stop],
    verbose=1
)



Num GPUs Available: 1
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[TabNet]: 32 features will be used for decision steps.
Epoch 1/100
79/79 [==============================] - 20s 95ms/step - loss: 3.8259 - accuracy: 0.0221 - val_loss: 3.8080 - val_accuracy: 0.0230
Epoch 2/100
79/79 [==============================] - 5s 66ms/step - loss: 3.8072 - accuracy: 0.0227 - val_loss: 3.8069 - val_accuracy: 0.0222
Epoch 3/100
79/79 [==============================] - 5s 66ms/step - loss: 3.8068 - accuracy: 0.0222 - val_loss: 3.8068 - val_accuracy: 0.0215
Epoch 4/100
79/79 [==============================] - 5s 65ms/step - loss: 3.8063 - accuracy: 0.0233 - val_loss: 3.8069 - val_accuracy: 0.0228
Epoch 5/100
79/79 [==============================] - 5s 68ms/step - loss: 3.8061 - accuracy: 0.0228 - val_loss: 3.8069 - val_accuracy: 0.0223
Epoch 6/100
79/79 [==============================] - 6s 74ms/step - loss: 3.8059 - accuracy: 0.0229 - val_loss: 3.8076 - val_accuracy: 0.02

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

In [42]:
# ✅ Evaluation
y_pred_probs = tabnet_model.predict(X_valid_dict)  # shape: (num_samples, num_classes)
y_pred = np.argmax(y_pred_probs, axis=1)           # convert to class labels

accuracy = accuracy_score(y_valid, y_pred)
f1 = f1_score(y_valid, y_pred, average='weighted')

print(f'Validation Accuracy: {accuracy:.4f}')
print(f'Validation F1-Score: {f1:.4f}')

# ✅ Inference
def predict_project(waste_items, tools):
    waste_vec = mlb_waste.transform([waste_items])
    tools_vec = mlb_tools.transform([tools])
    X_input = np.hstack([waste_vec, tools_vec])
    X_input_dict = array_to_dict(X_input)
    
    pred_probs = tabnet_model.predict(X_input_dict)
    pred_label = np.argmax(pred_probs, axis=1)[0]   # convert to class index
    project_name = target_encoder.inverse_transform([pred_label])[0]
    
    return project_name


# Example
new_waste_items = ["Ribbon Scrap", "Rope Piece"]
new_tools = ["Glue", "Scissors"]

predicted_project = predict_project(new_waste_items, new_tools)
print(f"Suggested DIY Project: {predicted_project}")

625/625 [==============================] - 12s 18ms/step
Validation Accuracy: 0.0224
Validation F1-Score: 0.0019
1/1 [==============================] - 0s 62ms/step
Suggested DIY Project: Creative Bracelet


d:\Anaconda\envs\py_gpu\lib\site-packages\sklearn\preprocessing\_label.py:909: UserWarning: unknown class(es) ['Glue'] will be ignored
  warnings.warn(
